In [56]:
import pandas as pd
import os
import re



In [57]:
columns = ['AUTHOR_NAME', 'PUB_TITLE', 'PUB_YEAR', 'PUB_AUTHORS', "PUB_NUMBER_AUTHORS", 'PUB_TYPE',
       'PUB_PUBLISHER', 'PUB_ISBN', 'PUB_DOI', 'PUB_CITATIONS',
       'JOU_RATING_VHB', 'JOU_RATING_SCIMAGO', 'UPDATED', 'YEAR_STORED',
       'SITE']
df_all_sites = pd.DataFrame(columns=columns)

In [58]:
files_in_data = os.listdir("../data")

p = re.compile('[0-9]{4}')
year_list = [ s for s in files_in_data if p.match(s)]

In [59]:
year = year_list[0]

In [ ]:
def list_value_counts(list_string):
    clean_list = list_string.replace("'", "").split(",")
    return len(clean_list)

In [109]:
for year in year_list:

    files_in_year = os.listdir(f"../data/{year}")

    q = re.compile('publications_[a-z-]+.csv')
    publication_file_list = [ s for s in files_in_year if q.match(s)]
    
    for publication_file_name in publication_file_list:
        site = publication_file_name.replace('publications_', '').replace('.csv', '')
        
        site_publications = pd.read_csv(f'../data/{year}/{publication_file_name}')
        
        site_publications["YEAR_STORED"] = year
        site_publications["SITE"] = site

        site_publications["PUB_NUMBER_AUTHORS"] = site_publications["PUB_AUTHORS"].apply(list_value_counts)

        df_all_sites = df_all_sites.append(site_publications)

In [119]:
df_all_sites

,AUTHOR_NAME,JOU_RATING_SCIMAGO,JOU_RATING_VHB,PUB_AUTHORS,PUB_CITATIONS,PUB_DOI,PUB_ISBN,PUB_NUMBER_AUTHORS,PUB_PUBLISHER,PUB_TITLE,PUB_TYPE,PUB_YEAR,SITE,UPDATED,YEAR_STORED
0,Oliver Keßling,NaN,NaN,['O Keßling'],6.0,NaN,NaN,1,NaN,AKF–Neues industrielles additives Verfahren,NaN,2014.0,campus-horb,2022-03-04,2021
1,Jürgen Gundrum,NaN,NaN,['J Gundrum'],1.0,NaN,NaN,1,NaN,Quarzsensoren für Viskositätsmessungen,NaN,1996.0,campus-horb,2022-03-04,2021
2,Jens Häcker,179.0,B,"['DD Freyw', 'Fredrik Engelhardt', 'Jens Haeck...",12.0,10.1080/00207540110116309,NaN,6,International Journal of Production Research,Robust manufacturing inspection and classifica...,Article,2002.0,campus-horb,NaN,2021
3,Jens Häcker,NaN,NaN,"['J Häcker', ' F Lehmann']",1.0,NaN,NaN,2,NaN,Scaling laws for biomometic flapping wing robo...,NaN,NaN,campus-horb,2022-03-04,2021
4,Jens Häcker,NaN,NaN,"['S Brückner', ' J Häcker', ' R Kornmann', ' B...",1.0,NaN,NaN,4,AIAA Aerodynamic …,The ISD parawing recovery system for high-alti...,NaN,2005.0,campus-horb,2022-03-04,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2526,Stefan Höflacher,NaN,NaN,"['FW Wagner', ' E Wenger', ' S Höflacher']",NaN,NaN,NaN,3,NaN,Strukturmerkmale des Optimierungsproblems,NaN,1986.0,stuttgart,2022-03-25,2022
2527,Stefan Höflacher,NaN,NaN,"['FW Wagner', ' E Wenger', ' S Höflacher']",NaN,NaN,NaN,3,NaN,Optimierungsprobleme bei linearem konstantem T...,NaN,1986.0,stuttgart,2022-03-25,2022
2528,Stefan Höflacher,NaN,NaN,"['S Höflacher', ' R Hufnagel', ' M Jaquemoth… ']",NaN,NaN,NaN,3,NaN,OIKOS 2010–Haushalte und Familien im Modernisi...,NaN,2007.0,stuttgart,2022-03-25,2022
2529,Stefan Höflacher,NaN,NaN,"['FW Wagner', ' E Wenger', ' S Höflacher']",NaN,NaN,NaN,3,NaN,Optimierungsprobleme bei regulärem Grenzsteuer...,NaN,1986.0,stuttgart,2022-03-25,2022


In [61]:
df_all_sites.to_csv(f'../data/publications_all_sites_all_years.csv', index=False)

In [70]:
len(df_all_sites[df_all_sites["YEAR_STORED"] == "2022"])

5210